In [59]:
# !pip install tqdm
# !pip install openpyxl
# !pip install deep_translator
# !pip install NLPAug
# !pip install gensim
!pip install torchvision 

Defaulting to user installation because normal site-packages is not writeable


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


In [60]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import re
import nltk
from nltk.tokenize import word_tokenize
from string import punctuation
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer #word lemma class
from deep_translator import GoogleTranslator # import google translator.
from tqdm import tqdm 
tqdm.pandas()
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.layers import Conv1D,Input, Bidirectional, LSTM, Dense, SpatialDropout1D, Input, Dropout,Embedding,Concatenate, TimeDistributed , BatchNormalization,Flatten,Activation
from tensorflow.keras import Model
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from sklearn.metrics import classification_report,accuracy_score
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.sequence import pad_sequences
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
import pickle
lemma = WordNetLemmatizer()
Gtrans= GoogleTranslator(source='auto', target='english')
#initialize and download words,stopword and wordnet word corpus.
words = set(nltk.corpus.words.words())
nltk.download('stopwords')
nltk.download('words')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [61]:
ticket=pd.read_excel('input_data automated ticket dataset.xlsx')
print(ticket.head(2))

  Short description                                        Description  \
0       login issue  -verified user details.(employee# & manager na...   
1           outlook  \r\n\r\nreceived from: hmjdrvpb.komuaywn@gmail...   

              Caller Assignment group  
0  spxjnwir pjlcoqds            GRP_0  
1  hmjdrvpb komuaywn            GRP_0  


In [62]:
ticket['Description'].fillna(ticket['Short description'],inplace=True)

In [63]:
# Define function for cleaning.
def dataCleaner(text):
    
    #remove x000d from text.
    text=text.replace('_x000D_',' ')
    
    #remove ? from text.
    text=text.replace('?',' ')
    
    #correct acct to account.
    text=text.replace('acct','account')
    
    #correct fw to forward.
    text=text.replace('fw:','forward')
    
    #correct fw to forward.
    text=text.replace(' fw ',' forward ')
    
    #remove (employee# & manager name)    
    #text=text.replace('(employee# & manager name)',' ')
        
    #remove subject:    
    #text=text.replace('subject:',' ')    
    
    #Remove received from: email
   
    
    #Remove hex chars
    lst = re.findall('==pcap 1 hex s==[\w\W]*==pcap 1 hex e==', text)
    for i in lst:
        text=text.replace(i,' ')
        
    #Remove ascii chars
    lst = re.findall('==pcap 1 ascii s==[\w\W]*==pcap 1 ascii e==', text)
    for i in lst:
        text=text.replace(i,' ')
        
    #Remove correlation data 
    lst = re.findall('\[correlation_data\](.*\n){2}', text)
    for i in lst:
        text=text.replace(i,' ')
       
    #Remove [correlation_data]
    lst = re.findall('\[correlation_data\]', text)
    for i in lst:
        text=text.replace(i,' ')
        
    #Remove [no entry] 
    lst = re.findall('\[no entry\]', text)
    for i in lst:
        text=text.replace(i,' ')
        
    #Remove ascii packet(s):
    text=text.replace('ascii packet(s):',' ')
    
    #Remove hex packet(s)::
    text=text.replace('hex packet(s):',' ')
        
    # Removing white spaces and trimmign the space around words.
    text = re.sub(' +', ' ', text.strip())
    
    #removee tabs
    text=text.replace('\t',' ')
    
    text=text.replace('\r',' ')
    
    #Remove new lines.
    text=text.replace('\n',' ')
    
    # We will use google translate to translate the sentence to english.
    if(len(text)>4000):
        n=3000
        GtransStrings = []
        for index in range(0, len(text), n):
            separated_strings = text[index : index + n]
            GtransStrings.append(Gtrans.translate(separated_strings))
        text=" ".join(GtransStrings)
        
    else:
        if(text!=''):
            text=Gtrans.translate(text)
            
    # Changing to lower case all the words.
    text = text.lower()
    text = text.split()
    
    # Removing english stop words.
    text = [word.strip() for word in text if not word in set(stopwords.words('english'))]
    
    # lemmatizing the words to their base forms
    text = [lemma.lemmatize(word) for word in text]
    
    # Joining back the text.
    text = ' '.join(text)
   
    return text

In [64]:
# Now lets clean all the text columns in the dataframe.
# ticket['CleanedDescription'] = ticket['Description'].progress_apply(dataCleaner)

In [65]:
# ticket.to_csv('./semicleaned2.csv',index=False)
ticket = pd.read_csv('./semicleaned2.csv')

In [66]:
ticket.isna().sum()

Short description      8
Description            0
Caller                 0
Assignment group       0
CleanedDescription    59
dtype: int64

In [67]:
ticket.drop(['Caller','Description','Short description'],axis=1,inplace=True)

In [68]:
ticket.dropna(inplace=True)

In [69]:
ticket['Group_Mod'] = ticket['Assignment group'].apply(lambda x: 'GRP_X' if x != 'GRP_0' else x)

In [70]:
temp=ticket[ticket['Group_Mod']!='GRP_0'].copy()
temp.head()

,Assignment group,CleanedDescription,Group_Mod
6,GRP_1,event: critical:hostname_221.company.com value...,GRP_X
17,GRP_3,"undocking pc , screen come back",GRP_X
32,GRP_4,received from: kxsceyzo.naokumlb@gmail.com gen...,GRP_X
43,GRP_5,received from: yisohglr.uvteflgb@gmail.com hi ...,GRP_X
47,GRP_6,received from: monitoring_tool@company.com job...,GRP_X


In [71]:
temp1=temp.copy()

In [72]:
aug = naw.SynonymAug(aug_src='wordnet', model_path=None, name='Synonym_Aug', aug_min=1, aug_max=10, aug_p=0.3,  
                     stopwords=None, tokenizer=None, reverse_tokenizer=None, stopwords_regex=None, force_reload=False, 
                     verbose=0)
temp1['CleanedDescription'] = temp1['CleanedDescription'].progress_apply(aug.augment)

100%|█████████████████████████████████████████████████████████████████████████████| 4468/4468 [00:12<00:00, 364.61it/s]


In [73]:
print(len(temp))
print(len(temp1))

4468
4468


In [74]:
newTemp1=pd.concat([temp,temp1])
print(len(newTemp1))

8936


In [75]:
XSmall=newTemp1['CleanedDescription']
ySmall=newTemp1['Assignment group']

In [76]:
print(XSmall.shape)
print(ySmall.shape)

(8936,)
(8936,)


In [77]:
XFull=ticket['CleanedDescription']
yFull=ticket['Group_Mod']

In [78]:
print(XFull.shape)
print(yFull.shape)

(8441,)
(8441,)


In [79]:
tokenizer = Tokenizer(lower=False)
tokenizer.fit_on_texts(ticket['CleanedDescription'])

word_index = tokenizer.word_index
vocab_size = len(tokenizer.word_index) + 1
print("Vocabulary Size :", vocab_size)
print("Word Indices: ", list(word_index.keys())[9])

Vocabulary Size : 18764
Word Indices:  tool


In [80]:
FullEncoder=LabelEncoder()
FullEncoder.fit(yFull)

LabelEncoder()

In [81]:
#max_len = max(ticket['detailLen'])
max_len=200
print(max_len)

200


In [82]:
XFull = pad_sequences(tokenizer.texts_to_sequences(XFull), maxlen = max_len)

In [83]:
yFull=FullEncoder.transform(yFull)

In [84]:
EMBEDDING_DIM = 300

In [85]:
# Generate word embeddings
# Construct the embedding weight matrix using the word embeddings via the tokenizer
embeddings_index = dict()
BATCH_SIZE = 1024
EPOCHS = 15

f = open('./glove.6B.300d.txt', encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

embedding_matrix = np.zeros((vocab_size, EMBEDDING_DIM))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
            
print('Found %s word vectors.' %len(embeddings_index))

Found 400000 word vectors.


In [86]:
Xf_train, Xf_test, yf_train, yf_test =  train_test_split(XFull, yFull, test_size = 0.30)

In [87]:
#defining callbacks
callbacks = [    
    ModelCheckpoint('./model-{val_loss:.2f}.h5', monitor="val_loss", verbose=1, save_best_only=True, save_weights_only=False),
    EarlyStopping(monitor="val_loss", patience=5),
    ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=5, min_lr=1e-6, verbose=1)
]

In [88]:
model = Sequential()
model.add(Embedding(vocab_size, EMBEDDING_DIM, weights = [embedding_matrix], input_length = max_len, trainable = False))
model.add(Bidirectional(LSTM(128, return_sequences=True, input_shape=(None, 1))))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(128)))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1, activation='sigmoid'))
#compile model
LR = 0.01
model.compile(loss = 'binary_crossentropy', optimizer=Adam(learning_rate = LR),metrics = ['accuracy'])

In [89]:
model.fit(Xf_train, yf_train, batch_size = 32, epochs = 10, validation_data = (Xf_test, yf_test), callbacks = callbacks)

Epoch 1/10
185/185 [==============================] - 111s 580ms/step - loss: 0.4918 - accuracy: 0.7502 - val_loss: 0.4660 - val_accuracy: 0.8069

Epoch 00001: val_loss improved from inf to 0.46598, saving model to .\model-0.47.h5
Epoch 2/10
185/185 [==============================] - 109s 588ms/step - loss: 0.3885 - accuracy: 0.8240 - val_loss: 0.4037 - val_accuracy: 0.8141

Epoch 00002: val_loss improved from 0.46598 to 0.40365, saving model to .\model-0.40.h5
Epoch 3/10
185/185 [==============================] - 111s 599ms/step - loss: 0.3821 - accuracy: 0.8257 - val_loss: 0.3871 - val_accuracy: 0.8125

Epoch 00003: val_loss improved from 0.40365 to 0.38708, saving model to .\model-0.39.h5
Epoch 4/10
185/185 [==============================] - 113s 609ms/step - loss: 0.3210 - accuracy: 0.8600 - val_loss: 0.3784 - val_accuracy: 0.8322

Epoch 00004: val_loss improved from 0.38708 to 0.37840, saving model to .\model-0.38.h5
Epoch 5/10
185/185 [==============================] - 116s 629ms

In [90]:
Xs_train, Xs_test, ys_train, ys_test =  train_test_split(XSmall, ySmall, test_size = 0.30)

In [91]:
#initialize the vectorizer
vectorizer = TfidfVectorizer(min_df=5 ,use_idf=True,analyzer='word', token_pattern=r'\w{1,}')
# We will fit on whole data set do that there shouldnt be any case where a word is present in train set and not in test.
vectorizer.fit(XSmall)
Xs_train = vectorizer.transform(Xs_train)
Xs_test = vectorizer.transform(Xs_test)

In [92]:
Xs_train=Xs_train.toarray()
Xs_test=Xs_test.toarray()

In [93]:
smallEncoder=LabelEncoder()
smallEncoder.fit(ySmall)
ys_train=smallEncoder.transform(ys_train)
ys_test=smallEncoder.transform(ys_test)

In [94]:
n_outputs = 1
n_inputs = Xs_train.shape[1]
model = Sequential()
# We will keep input shape as 200, the size of our feature list.
model.add(Dense(512, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
model.add(Dense(n_outputs, activation='sigmoid'))
# compile model with loss as binary_crossentropy and metrics as accuracy
model.compile(loss='binary_crossentropy', metrics = ['accuracy'],optimizer='adam')

#Train/fit the model
model.fit(Xs_train, ys_train, batch_size=8, epochs=25,verbose=1)

Epoch 1/25
782/782 [==============================] - 7s 8ms/step - loss: -10957.2334 - accuracy: 0.0312
Epoch 2/25
782/782 [==============================] - 6s 8ms/step - loss: -77637.3281 - accuracy: 0.0312
Epoch 3/25
782/782 [==============================] - 6s 8ms/step - loss: -199490.7500 - accuracy: 0.0312
Epoch 4/25
782/782 [==============================] - 7s 8ms/step - loss: -365813.4062 - accuracy: 0.0312
Epoch 5/25
782/782 [==============================] - 7s 9ms/step - loss: -570401.5000 - accuracy: 0.0312
Epoch 6/25
782/782 [==============================] - 7s 9ms/step - loss: -809570.9375 - accuracy: 0.0312
Epoch 7/25
782/782 [==============================] - 7s 8ms/step - loss: -1081362.2500 - accuracy: 0.0312
Epoch 8/25
782/782 [==============================] - 7s 8ms/step - loss: -1386484.7500 - accuracy: 0.0312
Epoch 9/25
782/782 [==============================] - 6s 8ms/step - loss: -1723319.3750 - accuracy: 0.0312
Epoch 10/25
782/782 [========================

In [95]:
param_grid_lr = {'penalty': ['l1','l2'], 'C': [0.001,0.01,0.1,1,10,100],'multi_class':['multinomial','auto']
               ,'solver':['newton-cg', 'lbfgs', 'liblinear', 'sag'],'n_jobs':[-1],'max_iter':[1000]}
grid_lr = GridSearchCV(LogisticRegression() , param_grid_lr, cv = 2, n_jobs=-1, scoring='accuracy',verbose=1)
# Training/Fitting Grid for Logistic Regression
grid_lr.fit(Xs_train,ys_train)
pickle.dump(grid_lr.best_estimator_, open('./lr_model.sav', 'wb'))

Fitting 2 folds for each of 96 candidates, totalling 192 fits


C:\Users\Dell\AppData\Roaming\Python\Python36\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=2.
  % (min_groups, self.n_splits)), Warning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


ValueError: Solver newton-cg supports only 'l2' or 'none' penalties, got l1 penalty.

In [ ]:
# Finding the best params which provide highest accuracy using different hyperparameter for Logistic regression model
print(grid_lr.best_params_)
print('LR accuracy on train data - ',grid_lr.score(Xs_train,ys_train))
print('LR accuracy on test data - ',grid_lr.score(Xs_test,ys_test))

In [ ]:
param_grid_nb = {} # Gaussian NB does not have hyper parameters to tune.
grid_nb = GridSearchCV(GaussianNB() , param_grid_nb, cv = 2, n_jobs=-1, scoring='accuracy',verbose=2)
# Training/Fitting Grid for NB
grid_nb.fit(Xs_train,ys_train)
pickle.dump(grid_nb.best_estimator_, open('./nb_model.sav', 'wb'))

In [ ]:
# Finding the best params which provide highest accuracy using different hyperparameter for Logistic regression model
print(grid_nb.best_params_)
print('NB accuracy on train data - ',grid_nb.score(Xs_train,ys_train))
print('NB accuracy on test data - ',grid_nb.score(Xs_test,ys_test))

In [ ]:
param_grid_knn = {'n_neighbors': [2,3,5],'algorithm': ['auto','brute'], 'n_jobs':[-1]}
grid_knn = GridSearchCV(KNeighborsClassifier() , param_grid_knn, cv =2, n_jobs=-1, scoring='accuracy',verbose=2)
# Training/Fitting Grid for KNN
grid_knn.fit(Xs_train,ys_train)
pickle.dump(grid_knn.best_estimator_, open('./knn_model.sav', 'wb'))

In [ ]:
# Finding the best params which provide highest accuracy using different hyperparameter for Logistic regression model
print(grid_knn.best_params_)
print('KNN accuracy on train data - ',grid_knn.score(Xs_train,ys_train))
print('KNN accuracy on test data - ',grid_knn.score(Xs_test,ys_test))

In [ ]:
param_grid_dt = {'criterion': ['gini','entropy'],'max_depth': [3,5,7], 'max_features':['auto','log2']}
grid_dt = GridSearchCV(DecisionTreeClassifier() , param_grid_dt, cv = 2, n_jobs=-1, scoring='accuracy',verbose=2)
# Training/Fitting Grid for DT
grid_dt.fit(Xs_train,ys_train)
pickle.dump(grid_dt.best_estimator_, open('./dt_model.sav', 'wb'))

In [ ]:
# Finding the best params which provide highest accuracy using different hyperparameter for Logistic regression model
print(grid_dt.best_params_)
print('DT accuracy on train data - ',grid_dt.score(Xs_train,ys_train))
print('DT accuracy on test data - ',grid_dt.score(Xs_test,ys_test))

In [ ]:
pickle.dump(tokenizer, open('./tokenizer.sav', 'wb'))
pickle.dump(vectorizer, open('./vectorizer.sav', 'wb'))
pickle.dump(smallEncoder, open('./smallEncoder.sav', 'wb'))
pickle.dump(FullEncoder, open('./FullEncoder.sav', 'wb'))

In [ ]:
model1 = keras.models.load_model('./model-0.37.h5')
model2 = pickle.load(open('./lr_model.sav', 'rb'))
max_len = 200
tokenizer = pickle.load(open('./tokenizer.sav', 'rb'))
vectorizer = pickle.load(open('./vectorizer.sav', 'rb'))
smallEncoder = pickle.load(open('./smallEncoder.sav', 'rb'))
FullEncoder = pickle.load(open('./FullEncoder.sav', 'rb'))

In [ ]:
def getGroupAssignment(text):
    text=dataCleaner(text)
    if text.strip() == '' :
        return 'GRP_0'
    X1=pad_sequences(tokenizer.texts_to_sequences([text]), maxlen = max_len)
    model1Pred=FullEncoder.inverse_transform(np.round(model1.predict(X1),0).astype(int))
    
    if model1Pred=='GRP_0':
        return 'GRP_0'
    else:
        X2=vectorizer.transform([text])
        X2=X2.toarray()
        try:
            model2Pred=smallEncoder.inverse_transform(model2.predict(X2))
            model2Pred=model2Pred[0]
        except:
            model2Pred = 'GRP_0'
        return model2Pred
    
    return 'GRP_0'

In [ ]:
# Function getGroupAssignment is used to predict for new text.
getGroupAssignment('when undocking pc , screen will not come back')